In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

2023-03-08 11:18:51.960849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 11:18:52.183213: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-08 11:18:52.190487: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 11:18:52.190509: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
import os
import s3fs

In [3]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [4]:
BUCKET="gschone"
fs.ls(BUCKET)

['gschone/MNIST_DNN.ipynb',
 'gschone/data_classif.rds',
 'gschone/data_test.csv',
 'gschone/data_train.csv',
 'gschone/don_class.rds',
 'gschone/resCLASUP.RDS',
 'gschone/test.rds']

In [17]:
BUCKET = "gschone"
FILE_KEY_S3 = "data_test.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
   df_test = pd.read_csv(file_in, sep=",")

In [11]:
BUCKET = "gschone"
FILE_KEY_S3 = "data_train.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
   df_train = pd.read_csv(file_in, sep=",")

In [25]:
y_train=df_train["Y"]
y_test=df_test["Y"]

In [33]:
x_train= df_train.drop('Y', axis=1)
x_test= df_test.drop('Y', axis=1)
x_test

,Home_buildUpPlaySpeed,Home_buildUpPlayPassing,Home_chanceCreationPassing,Home_chanceCreationCrossing,Home_chanceCreationShooting,Home_defencePressure,Home_defenceAggression,Home_defenceTeamWidth,Away_buildUpPlaySpeed,Away_buildUpPlayPassing,...,H10_overall_rating_ns_3,A10_overall_rating_ns_1,A10_overall_rating_ns_2,A10_overall_rating_ns_3,H11_overall_rating_ns_1,H11_overall_rating_ns_2,H11_overall_rating_ns_3,A11_overall_rating_ns_1,A11_overall_rating_ns_2,A11_overall_rating_ns_3
0,-0.142604,1.333618,0.799448,1.062186,0.156257,0.018986,1.733190,-0.633128,0.314461,0.947733,...,-0.322656,0.579401,0.372723,-0.135779,0.144940,0.540407,-0.315237,0.580786,0.362564,-0.115526
1,-0.599670,-1.367576,-0.102864,0.123859,1.518962,-0.480648,0.803866,-1.188503,0.862940,-1.753460,...,-0.137387,0.290350,0.341584,0.356312,0.227329,0.519847,-0.303244,0.555619,0.337184,0.019042
2,0.862940,-1.753460,-1.907487,-1.283631,-1.660684,-1.479916,-1.054783,-0.077753,0.131635,-1.753460,...,0.164657,0.022580,0.557073,-0.321388,0.553422,0.337110,0.030436,0.264064,0.508756,-0.301022
3,1.228592,1.333618,1.701760,1.062186,1.518962,2.517155,1.733190,2.143748,1.228592,0.561848,...,-0.217852,0.584433,0.355052,-0.073369,0.415672,0.455384,-0.259688,0.511052,0.332564,0.097328
4,0.314461,1.333618,1.701760,1.062186,-0.297978,-0.480648,-1.519446,-1.188503,1.228592,1.333618,...,-0.137387,0.146667,0.541958,-0.312668,0.415672,0.455384,-0.259688,0.434351,0.445473,-0.251484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,-0.873909,-0.981691,-0.824713,-0.251472,-0.207131,1.717741,1.175596,-0.744203,0.588700,-1.444753,...,-0.137387,0.546608,0.396907,-0.187959,0.415672,0.455384,-0.259688,0.434351,0.445473,-0.251484
453,0.405874,1.179264,1.431066,0.311524,-0.116284,-0.880355,-1.612378,-0.299903,-0.142604,-0.132744,...,-0.304951,0.079282,0.553459,-0.319303,-0.030074,0.548622,-0.320029,0.556268,0.384614,-0.170049
454,0.405874,-0.827337,-1.095406,0.030026,-0.479672,0.718474,0.618001,1.032998,1.320005,-1.599106,...,-0.288107,0.146667,0.541958,-0.312668,0.588505,0.363739,-0.120217,0.511052,0.332564,0.097328
455,0.223048,-1.830637,-1.727024,-0.908300,0.792186,-0.280794,-0.683054,0.366548,-0.234018,-0.441452,...,-0.137387,0.314273,0.494071,-0.284400,0.096226,0.358920,0.543011,0.108161,0.549116,-0.325402


In [37]:
num_layer1=100
num_layer2=100
model=keras.models.Sequential(name='DNN')
model.add(keras.layers.Input((1283,1),name='Couche_entree'))
model.add(keras.layers.Dense(num_layer1,activation='relu',name='Couche_cachee1'))
model.add(keras.layers.Dense(num_layer2,activation='relu',name='Couche_cachee2'))
model.add(keras.layers.Dense(3, activation='softmax', name='Couche_sortie'))


In [40]:
model.summary()

Model: "DNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Couche_cachee1 (Dense)      (None, 1283, 100)         200       
                                                                 
 Couche_cachee2 (Dense)      (None, 1283, 100)         10100     
                                                                 
 Couche_sortie (Dense)       (None, 1283, 3)           303       
                                                                 
Total params: 10,603
Trainable params: 10,603
Non-trainable params: 0
_________________________________________________________________
